In [78]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import re

# Leyendo csv de los datos historicos

In [79]:
# precio de etherium por dia desde el 24 de dic de 2016 hasta el 24 de enero de 2020
csv = pd.read_csv('Ethereum Historical Data - Investing.com.csv')
csv['Date'] = pd.to_datetime(csv['Date'], infer_datetime_format=True)
csv_df = csv.sort_values(by='Date').reset_index()
csv_df.tail(5)

,index,Date,Price,Open,High,Low,Vol.,Change %
1122,4,2020-01-20,166.87,166.92,169.24,161.61,11.30M,-0.03%
1123,3,2020-01-21,169.41,166.87,169.79,165.53,11.52M,1.52%
1124,2,2020-01-22,168.13,169.41,171.27,166.19,11.30M,-0.76%
1125,1,2020-01-23,162.80,168.15,168.25,159.78,14.86M,-3.17%
1126,0,2020-01-24,163.08,162.80,164.27,155.65,15.33M,0.18%


# request de la api de cryptocompare

In [80]:
# ultimas 1000 transacciones registradas en el mercado de etherium por dia, todos los precios son en usd
response = requests.get('https://min-api.cryptocompare.com/data/v2/histoday?fsym=ETH&tsym=USD&limit=1126')
read = response.json()
high = []
low = []
open=[]
close = []
for i in read['Data']['Data']:
    high.append(i['high'])
    low.append(i['low'])
    open.append(i['open'])
    close.append(i['close'])

In [81]:
df = pd.DataFrame()
df['OPEN'] = open
df['HIGH'] = high
df['LOW'] = low
df['Close'] = close
df.tail(5)

,OPEN,HIGH,LOW,Close
1122,166.92,168.97,162.92,166.64
1123,166.64,171.09,166.56,169.09
1124,169.09,170.22,165.28,167.97
1125,167.97,164.02,160.41,162.51
1126,162.63,164.22,155.29,161.95


# csv yahoo finances

In [82]:
yah = pd.read_csv('ETH-USD.csv')
yah['Date'] = pd.to_datetime(yah['Date'], infer_datetime_format=True)
yah.tail(5)

,Date,Open,High,Low,Close,Adj Close,Volume
1122,2020-01-20,166.904129,169.114883,162.331604,167.120514,167.120514,11144529787
1123,2020-01-21,167.060471,169.910110,165.819717,169.697159,169.697159,9789369362
1124,2020-01-22,169.608536,171.008698,166.774231,168.294159,168.294159,9209827927
1125,2020-01-23,168.295700,168.312241,160.295319,162.928558,162.928558,10396868698
1126,2020-01-24,162.950241,164.293701,156.750778,163.630173,163.630173,10699618304


In [57]:
json_normalize(cg.get_coin_by_id('ethereum')).columns

Index(['id', 'symbol', 'name', 'asset_platform_id', 'block_time_in_minutes',
       'hashing_algorithm', 'categories', 'country_origin', 'genesis_date',
       'sentiment_votes_up_percentage',
       ...
       'developer_data.total_issues', 'developer_data.closed_issues',
       'developer_data.pull_requests_merged',
       'developer_data.pull_request_contributors',
       'developer_data.code_additions_deletions_4_weeks.additions',
       'developer_data.code_additions_deletions_4_weeks.deletions',
       'developer_data.commit_count_4_weeks',
       'developer_data.last_4_weeks_commit_activity_series',
       'public_interest_stats.alexa_rank',
       'public_interest_stats.bing_matches'],
      dtype='object', length=1354)

In [113]:
data = csv_df.join(df['Close'])
final = data.join(yah[['Adj Close', 'Volume']])
final2 = final.drop('index', axis=1)
final3 = final2[['Date','Price','Open','High','Low','Close','Adj Close','Vol.','Volume','Change %']]

final3.dtypes

Date         datetime64[ns]
Price                object
Open                 object
High                 object
Low                  object
Close               float64
Adj Close           float64
Vol.                 object
Volume                int64
Change %             object
dtype: object

In [115]:
final3['Price'] = final3['Price'].str.replace(',','')
final3['Open']=final3['Open'].str.replace(',','')
final3['High']=final3['High'].str.replace(',','')
final3['Low']=final3['Low'].str.replace(',','')

In [127]:
final4 = final3.astype({'Price':'float64'})
final4 = final4.astype({'Open':'float64'})
final4 = final4.astype({'High':'float64'})
final4 = final4.astype({'Low':'float64'})
final4.dtypes

Date         datetime64[ns]
Price               float64
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Vol.                 object
Volume                int64
Change %             object
dtype: object